In [1]:
import sys
import os
import logging
import time
import progressbar
import itertools
from itertools import izip_longest, izip, islice, tee, repeat

In [2]:
__author__='Maggie Ruimin Sun'

Reference source code: consolidate.py, scripted by Martin Aryee.
Source code can be found at https://github.com/aryeelab/umi/tree/3fef4c92becda4c2b4b6085555415f80c1dd858e

In [3]:
logger = logging.getLogger('root')

In [4]:
def read_bins(fastq_file):
    read_num = 0
    infile = open(fastq_file)
    reads = infile.readlines()
    Nreads = len(reads)/4
    infile.close()
    cur_molecular_id = ''
    i = 0
    while i < Nreads:
        read_header = reads[4*i]
        read_seq = reads[4*i + 1]
        read_qual = reads[4*i + 3]
        read_num += 1
        read_name, sample_id, molecular_id = read_header.strip().split(' ')
        read_qual = [str(ord(x)-33) for x in read_qual.strip()]
        read = [read_header.rstrip(), read_seq.rstrip(), '+', read_qual]
        i += 1
        if molecular_id == cur_molecular_id:
            bin_reads.append(read)
        else:
            if cur_molecular_id != '':
                yield cur_molecular_id, cur_sample_id, bin_reads
            cur_molecular_id = molecular_id
            cur_sample_id = sample_id
            bin_reads = [read]
    yield cur_molecular_id, cur_sample_id, bin_reads

In [5]:
def consolidate_position(bases, quals, min_qual, min_freq):
    num = {}
    qual = {}
    num['A'] = num['C'] = num['G'] = num['T'] = num['N'] = 0
    qual['A'] = qual['C'] = qual['G'] = qual['T'] = qual['N'] = 0
    for bb, qq in zip(bases, quals):
        qq = int(qq)
        if qq > min_qual:
            num[bb] += 1
        if qq > qual[bb]:
            qual[bb] = qq
    most_common_base = max(num.iterkeys(), key=(lambda key: num[key]))
    freq = float(num[most_common_base]) / len(bases)
    if freq > min_freq:
        return True, most_common_base, qual[most_common_base]
    else:
        return False, 'N', 0

In [6]:
def consolidate(fastq_file, consolidate_fastq_file, min_qual, min_freq):
    logger.info("Consolidating reads in %s", fastq_file)
    outfolder = os.path.dirname(consolidate_fastq_file)
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
        
    outfile = open(consolidate_fastq_file, 'w')
    bins = read_bins(fastq_file)

    
    num_input_reads = 0
    num_consolidate_reads = 0
    num_successes = 0
    num_bases = 0
    
    for cur_molecular_id, cur_sample_id, reads in bins:
        num_input_reads += len(reads)
        num_consolidate_reads += 1
        
        #For the same molecular id, group the bases in each read sequence by positoin.
        #Here, zip(*[list1, list2, ...]) make a transpose of [list1, list2, ...]
        read_bases = zip(*[list(read[1]) for read in reads])
        read_quals = zip(*[list(read[3]) for read in reads])
        consolidation_success, cons_seq, cons_qual = zip(*[consolidate_position(bases, quals,
                                                                               min_qual, min_freq,) 
                                                          for bases, quals, in zip(read_bases, read_quals)])
        num_successes += sum(consolidation_success)
        num_bases += len(consolidation_success)
        
        outfile.write('@%s_%d %s\n' % (cur_molecular_id, len(reads), cur_sample_id))
        outfile.write(''.join(cons_seq)+'\n+\n')
        outfile.write(''.join([chr(q+33) for q in cons_qual])+'\n')
#        if num_input_reads > 1000:
#            break
    outfile.close()
    print num_input_reads
    logger.info("Read %d input reads", num_input_reads)
    logger.info("Wrote %d consolidated reads to %s", num_consolidate_reads, consolidate_fastq_file)
    logger.info("Successfully consolidated %d bases out of %d (%.2f%%)", num_successes, num_bases, 100*float(num_successes)/num_bases)
    

In [7]:
def main():
    source =  '/home/yaneng/RSun/Data/qiagen-colon/'
    in_dir = 'umi_tagged/'
    out_dir = 'consolidated/'
    fastq_file1 = source + in_dir + 'QIAGEN-2959YJ_S2_L001_R1_001_umi.fq'
    fastq_file2 = source + in_dir + 'QIAGEN-2959YJ_S2_L001_R2_001_umi.fq'
    consolidate_fastq_file1 = source + out_dir + 'QIAGEN-2959YJ_S2_L001_R1_001_consolidated.fq'
    consolidate_fastq_file2 = source + out_dir + 'QIAGEN-2959YJ_S2_L001_R2_001_consolidated.fq'
    min_qual = 30
    min_freq = 0.9
    consolidate(fastq_file1, consolidate_fastq_file1, min_qual, min_freq)
    logger.info("Consolidation of reads in %s is successfully done.", fastq_file1)
    
    consolidate(fastq_file2, consolidate_fastq_file2, min_qual, min_freq)
    logger.info("Consolidation of reads in %s is successfully done.", fastq_file2)


In [8]:
if __name__ == '__main__':
    main()

1608349
1608349
